1 Individual source

In [ ]:
"""
1. Make new dirs: 
rootdir with region name, 
{region name}: fitting files,
others: for other files
log: logs for fitting
download the subdir of region name after that.

2. mosspectra and mosback in sas21
3. pn-spectra and pn_back in sas20

note: don't run mosspectra or pnspectra or backs at same time in same dir!

"""

In [ ]:
# for a new region 01r500
import os
from glob import glob

# SRCNAME1 = f''
# SRCNAME2 = f'RGH80'
# workpath = f'/data/yanling/XGAP-ABUN/data/data/{SRCNAME2}/reduction'


SRCNAME1 = f'ID828'
SRCNAME2 = f'SDSSTG828'
workpath = f'/data/yanling/XGAP-ABUN/data/data/{SRCNAME1}/reduction/{SRCNAME2}'


regname = 'R500_01'

savepath = f'{workpath}/{regname}'

ELO=700
EHI=1200


In [ ]:
f = open('tmp/esas_exspec.sh', 'w')
f.write(
f'''
#!/bin/bash

heainit
sasinit21
source {workpath}/set_chips_on.txt
source {workpath}/set_sas.txt
# #### MOS ####
# export M1=mos1S001
# export M2=mos2S002

# mkdir {regname}
# mkdir {regname}/{regname}
# mkdir {regname}/img
# mkdir {regname}/logs
# mkdir {regname}/diagnose

# # mosspectra eventfile=${{M1}}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/{regname}-${{M1}}.reg pattern=12 withsrcrem=yes maskdet=${{M1}}-bkgregtdet.fits masksky=${{M1}}-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_${{M1}}.log 
# # mosback inspecfile=${{M1}}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" 2>&1  |tee mosback_${{M1}}.log 
# mosspectra eventfile=${{M2}}-allevc.fits keepinterfiles=yes withregion=yes regionfile=spec_reg/{regname}-${{M2}}.reg pattern=12 withsrcrem=yes maskdet=${{M2}}-bkgregtdet.fits masksky=${{M2}}-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 2>&1 |tee {regname}/logs/mosspectra_${{M2}}.log
# mosback inspecfile=${{M2}}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}" 2>&1  |tee mosback_${{M2}}.log 

# for name in $M2
# do
# echo $name

# mv $name*-{ELO}-{EHI}* {regname}/img
# mv $name-fovt.pi R500_01/R500_01/$name-obj-{SRCNAME2}_R500_01.pi
# mv $name-bkg.pi R500_01/R500_01/$name-back-{SRCNAME2}_R500_01.pi
# mv $name*.qdp {regname}/diagnose
# mv $name*imt* {regname}/img
# mv $name*.arf {regname}/{regname}/$name-{SRCNAME2}_{regname}.arf
# mv $name*.rmf {regname}/{regname}/$name-{SRCNAME2}_{regname}.rmf
# mv $name*imspdet* {regname}/img
# done

#### PN ####
heainit
sasinit20
source {workpath}/set_chips_on.txt
source {workpath}/set_sas.txt
export name=pnS003
cp $name-allevc.fits $name-clean.fits
cp $name-allevcoot.fits $name-clean-oot.fits
cp $name-corevc.fits $name-corn.fits
cp $name-corevcoot.fits $name-corn-oot.fits
cp $name-bkgregtdet.fits sas20-pn-test/$name-bkg_region-sky.fits 
cp $name-bkgregtsky.fits sas20-pn-test/$name-bkg_region-det.fits 

for PAT in 0 4
do
pn-spectra prefix='S003' caldb=/data/yanling/sas20/esas-caldb region=spec_reg/R500_01-pnS003.reg mask=1 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 2>&1 | tee {regname}/logs/pnspectra_$PAT.log
pn_back prefix=S003 caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 2>&1 | tee {regname}/logs/pnback_$PAT.log

mkdir {regname}/img/pn_$PAT
mkdir {regname}/diagnose/pn_$PAT

mv $name*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv $name-obj.pi {regname}/{regname}/$name-$PAT-obj-{SRCNAME2}_{regname}.pi
mv $name-obj-oot.pi {regname}/{regname}/$name-$PAT-obj-oot-{SRCNAME2}_{regname}.pi
mv $name-back.pi {regname}/{regname}/$name-$PAT-back-{SRCNAME2}_{regname}.pi
mv $name*.qdp {regname}/diagnose/pn_$PAT
mv $name*imt* {regname}/img/pn_$PAT
mv $name*.arf {regname}/{regname}/$name-$PAT-{SRCNAME2}_{regname}.arf
mv $name*.rmf {regname}/{regname}/$name-$PAT-{SRCNAME2}_{regname}.rmf
mv $name*imspdet* {regname}/img/pn_$PAT

done
'''
)
f.close()

# cd {workpath}
# . codes/mosspectra.sh

2 All sources-specs

In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO=500
EHI=4000

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        for sregname in ['bkg', 'R500-01']:
            regname = f'{srcname}_{sregname}'

        
            f = open(f'mos_exspec_{sregname}.sh', 'w')
            f.write(
        f'''
heainit
sasinit21
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt


#### MOS ####
mkdir {regname}
mkdir {regname}/{regname}
mkdir {regname}/img
mkdir {regname}/logs
mkdir {regname}/diagnose

mosspectra eventfile={mos1file} keepinterfiles=yes withregion=yes regionfile=fk5_regs/{srcname}-mos1_{sregname}.txt pattern=12 withsrcrem=yes maskdet=mos1-bkgregtdet.fits masksky=mos1-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 > {regname}/logs/mosspectra_mos1.log 2>&1 
mosback inspecfile={mos1file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" > {regname}/logs/mosback_mos1.log 2>&1 
mosspectra eventfile={mos2file} keepinterfiles=yes withregion=yes regionfile=fk5_regs/{srcname}-mos2_{sregname}.txt pattern=12 withsrcrem=yes maskdet=mos2-bkgregtdet.fits masksky=mos2-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 > {regname}/logs/mosspectra_mos2.log 2>&1 
mosback inspecfile={mos2file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}" > {regname}/logs/mosback_mos2.log 2>&1 

for name in mos1 mos2
do
echo $name
mv $name*-{ELO}-{EHI}* {regname}/img
mv $name*.qdp {regname}/diagnose
mv $name*imt* {regname}/img
mv $name*imspdet* {regname}/img
done

mv mos1*-fovt.pi {regname}/{regname}/mos1S001-obj-{regname}.pi
mv mos1*-bkg.pi {regname}/{regname}/mos1S001-back-{regname}.pi
mv mos1*.arf {regname}/{regname}/mos1S001-{regname}.arf
mv mos1*.rmf {regname}/{regname}/mos1S001-{regname}.rmf

mv mos2*-fovt.pi {regname}/{regname}/mos2S002-obj-{regname}.pi
mv mos2*-bkg.pi {regname}/{regname}/mos2S002-back-{regname}.pi
mv mos2*.arf {regname}/{regname}/mos2S002-{regname}.arf
mv mos2*.rmf {regname}/{regname}/mos2S002-{regname}.rmf

        '''
        )
            f.close()


In [ ]:
import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO=700
EHI=1200

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        for sregname in ['bkg']: # 'bkg', 'R500-01'
            regname = f'{srcname}_{sregname}'

            with open(f'pn_exspec_{sregname}.sh', 'w') as f:
                f.write(
                f'''

#### PN ####
heainit
sasinit20
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt
export sname=pn
export name={pnname}
cp $sname*-allevc.fits $name-clean.fits
cp $sname*-allevcoot.fits $name-clean-oot.fits
cp $sname*-corevc.fits $name-corn.fits
cp $sname*-corevcoot.fits $name-corn-oot.fits
cp $sname-bkgregtdet.fits $name-bkg_region-sky.fits 
cp $sname-bkgregtsky.fits $name-bkg_region-det.fits 

for PAT in 0 4
do
pn-spectra prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb region=fk5_regs/{srcname}-pn_{sregname}.txt mask=1 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnspectra_$PAT.log 2>&1 
pn_back prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnback_$PAT.log 2>&1 

mkdir {regname}/img/pn_$PAT
mkdir {regname}/diagnose/pn_$PAT

mv pn*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv pn*-obj.pi {regname}/{regname}/pnS003-$PAT-obj-{regname}.pi
mv pn*-obj-oot.pi {regname}/{regname}/pnS003-$PAT-obj-oot-{regname}.pi
mv pn*-back.pi {regname}/{regname}/pnS003-$PAT-back-{regname}.pi
mv pn*.qdp {regname}/diagnose/pn_$PAT
mv pn*imt* {regname}/img/pn_$PAT
mv pn*.arf {regname}/{regname}/pnS003-$PAT-{regname}.arf
mv pn*.rmf {regname}/{regname}/pnS003-$PAT-{regname}.rmf
mv pn*imspdet* {regname}/img/pn_$PAT

done
''')


3 All sources-img

In [ ]:
#NOTE: Never use allevc-exchips events! The chips will be excluded in set_chips_on.txt!!!
import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO=500
EHI=4000

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        sregname = 'noreg'
        regname = f'{srcname}_{sregname}'

        
        with open(f'mos_exspec_{sregname}.sh', 'w') as f:
            f.write(
        f'''
heainit
sasinit21
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt


#### MOS ####
mkdir {regname}
mkdir {regname}/{regname}
mkdir {regname}/img
mkdir {regname}/logs
mkdir {regname}/diagnose

mosspectra eventfile={mos1file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=yes maskdet=mos1-bkgregtdet.fits masksky=mos1-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M1ON}}" -V=7 > {regname}/logs/mosspectra_mos1.log 2>&1 
mosback inspecfile={mos1file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M1ON}}" > {regname}/logs/mosback_mos1.log 2>&1 
mosspectra eventfile={mos2file} keepinterfiles=yes withregion=no pattern=12 withsrcrem=yes maskdet=mos2-bkgregtdet.fits masksky=mos2-bkgregtsky.fits elow={ELO} ehigh={EHI} ccds="${{M2ON}}" -V=7 > {regname}/logs/mosspectra_mos2.log 2>&1 
mosback inspecfile={mos2file.split('-')[0]}-fovt.pi elow={ELO} ehigh={EHI} ccds="${{M2ON}}" > {regname}/logs/mosback_mos2.log 2>&1 

for name in mos1 mos2
do
echo $name
mv $name*-{ELO}-{EHI}* {regname}/img
mv $name*.qdp {regname}/diagnose
mv $name*imt* {regname}/img
mv $name*imspdet* {regname}/img
done

mv mos1*-fovt.pi {regname}/{regname}/mos1S001-obj-{regname}.pi
mv mos1*-bkg.pi {regname}/{regname}/mos1S001-back-{regname}.pi
mv mos1*.arf {regname}/{regname}/mos1S001-{regname}.arf
mv mos1*.rmf {regname}/{regname}/mos1S001-{regname}.rmf

mv mos2*-fovt.pi {regname}/{regname}/mos2S002-obj-{regname}.pi
mv mos2*-bkg.pi {regname}/{regname}/mos2S002-back-{regname}.pi
mv mos2*.arf {regname}/{regname}/mos2S002-{regname}.arf
mv mos2*.rmf {regname}/{regname}/mos2S002-{regname}.rmf

        '''
        )


In [ ]:
import os
from glob import glob


rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'
ELO=500
EHI=4000

homepaths = glob(f'{rootdir}/SDSSTG*')

for homepath in homepaths:
    if os.path.isdir(homepath):
        os.chdir(homepath)
        srcname = homepath.split('/')[-1]
        mos1file = glob('mos1*-allevc.fits')[0]
        mos2file = glob('mos2*-allevc.fits')[0]
        pnfile = glob('pn*-allevc.fits')[0]
        pnname = pnfile.split('-')[0]
        pnname_suf = pnname[2:]

        
        sregname = 'noreg'
        regname = f'{srcname}_{sregname}'
        
        with open(f'pn_exspec_{sregname}.sh', 'w') as f:
            f.write(
                f'''

#### PN ####
heainit
sasinit20
source {homepath}/set_chips_on.txt
source {homepath}/set_sas.txt
export sname=pn
export name={pnname}
cp $sname*-allevc.fits $name-clean.fits
cp $sname*-allevcoot.fits $name-clean-oot.fits
cp $sname*-corevc.fits $name-corn.fits
cp $sname*-corevcoot.fits $name-corn-oot.fits
cp $sname-bkgregtdet.fits $name-bkg_region-sky.fits 
cp $sname-bkgregtsky.fits $name-bkg_region-det.fits 

for PAT in 0 4
do
pn-spectra prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb mask=1 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnspectra_$PAT.log 2>&1 
pn_back prefix='{pnname_suf}' caldb=/data/yanling/sas20/esas-caldb diag=0 elow={ELO} ehigh={EHI} pattern=$PAT quad1=1 quad2=1 quad3=1 quad4=1 > {regname}/logs/pnback_$PAT.log 2>&1 

mkdir {regname}/img/pn_$PAT
mkdir {regname}/diagnose/pn_$PAT

mv pn*-{ELO}-{EHI}* {regname}/img/pn_$PAT
mv pn*-obj.pi {regname}/{regname}/pnS003-$PAT-obj-{regname}.pi
mv pn*-obj-oot.pi {regname}/{regname}/pnS003-$PAT-obj-oot-{regname}.pi
mv pn*-back.pi {regname}/{regname}/pnS003-$PAT-back-{regname}.pi
mv pn*.qdp {regname}/diagnose/pn_$PAT
mv pn*imt* {regname}/img/pn_$PAT
mv pn*.arf {regname}/{regname}/pnS003-$PAT-{regname}.arf
mv pn*.rmf {regname}/{regname}/pnS003-$PAT-{regname}.rmf
mv pn*imspdet* {regname}/img/pn_$PAT

done
''')


# 3 run in parallel

In [ ]:
#!/bin/bash
find . -mindepth 1 -maxdepth 1 -type d -name 'SDSSTG*' -print0 | parallel -0 -j+0 'cd {} && chmod +x esas_exspec_R500-01.sh && heainit && sasinit21 && source set_sas.txt && source set_chips_on.txt && ./esas_exspec_R500-01.sh'

# 4 download from server

4.1 download fitting dirs

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os


my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
server_rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'

regname = 'R500-01'
# read ids
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS/get_nh'
f = pd.read_csv(f'{datapath}/basics_allsources.csv')
ids = f['ID']

# ids = ['SDSSTG8050']
# ids = ['SDSSTG9178', 'SDSSTG3669', 'SDSSTG22635', 'SDSSTG39344', 'SDSSTG11320','SDSSTG6159', 'SDSSTG1601','SDSSTG28674','SDSSTG8102', 'SDSSTG11631', 'SDSSTG16386','SDSSTG10159']
# ids = ['SDSSTG10842']
for id in tqdm(ids):
    savepath = f'{my_rootdir}/{id}'
    # files_exist = glob(f'{savepath}/fit_23*/figs/*GD*pdf')
    # if not files_exist:
    # print(savepath)
    # os.makedirs(savepath, exist_ok=True)
    os.system(f'scp -P3333 -r yanling@101.6.240.92:{server_rootdir}/{id}/fit_spex_231115/\*/\*GD\* {savepath}/fit_spex_231115')
    # os.system(f'cp -r {savepath}/fit_spex_231115/fit_spex_231115/ {savepath}/fit_spex')
    # os.system(f'rm -r  {savepath}/fit_spex_231115')
    # os.system(f'mv {savepath}fit_spex_231115 {savepath}/fit_spex_231115')

  2%|▏         | 1/48 [00:31<24:41, 31.52s/it]scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexsys0002484211.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexpar0002488185.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexmod0002785973.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexsys0002488185.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexsys0002785973.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexdat0002484211.dum/"): No such file or directory
scp: remote readdir("/data/yanling/XGAP-ABUN/data/alldata/XGAP/SDSSTG10842/fit_spex_231115/spexd

In [ ]:
import pandas as pd
import numpy as np
import os

my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
server_rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'

regname = 'R500-01'
# read ids
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS/get_nh'
f = pd.read_csv(f'{datapath}/basics_allsources.csv')
ids = f['ID']
# print(np.array(ids))
ids = ['SDSSTG22635', 'SDSSTG6159', 'SDSSTG9370', 'SDSSTG1601', 'SDSSTG28674', 'SDSSTG11320', 'SDSSTG8102', 'SDSSTG3669', 'SDSSTG11631', 'SDSSTG9178']
for id in ids:
    savepath = f'{my_rootdir}/{id}'
    print(savepath)
    os.makedirs(savepath, exist_ok=True)
    os.system(f'scp -P3333 -r yanling@101.6.240.92:{server_rootdir}/{id}/fit_231* {savepath}')

In [ ]:
import pandas as pd
import numpy as np
import os

my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
server_rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'

os.system(f'scp -P3333 -r yanling@101.6.240.92:/data/yanling/XGAP-ABUN/data/alldata/XGAP/dats/*.csv {my_rootdir}/../dats')

In [ ]:
spex_set1 = ['SDSSTG8050','SDSSTG10842','SDSSTG885','SDSSTG1011','SDSSTG4654']
spex_set2 = ['SDSSTG1695','SDSSTG15641','SDSSTG3128'
,'SDSSTG12349','SDSSTG16150']
spex_set3 = ['SDSSTG3460','SDSSTG9771','SDSSTG10094'
,'SDSSTG5742','SDSSTG6058','SDSSTG1162']
spex_set4 = ['SDSSTG15354','SDSSTG2424'
,'SDSSTG46701','SDSSTG35976','SDSSTG9399','SDSSTG4436']
spex_set5 = ['SDSSTG9647'
,'SDSSTG4936','SDSSTG828','SDSSTG15776','SDSSTG9370','SDSSTG21128']
spex_set6 = ['SDSSTG1398','SDSSTG40241','SDSSTG11844','SDSSTG4047','SDSSTG3513'
,'SDSSTG16393']

In [ ]:
# gdem

spex_set1 = ['SDSSTG10842','SDSSTG885','SDSSTG1011','SDSSTG4654', 'SDSSTG8050']
spex_set2 = ['SDSSTG3128','SDSSTG12349','SDSSTG16150', 'SDSSTG10094','SDSSTG5742','SDSSTG6058','SDSSTG1162']
spex_set4 = ['SDSSTG2424'
,'SDSSTG46701','SDSSTG35976','SDSSTG9399','SDSSTG4436']
spex_set5 = ['SDSSTG9647'
,'SDSSTG4936','SDSSTG15776','SDSSTG21128']
spex_set6 = ['SDSSTG11844','SDSSTG4047','SDSSTG3513'
,'SDSSTG16393']

spex_set = ['SDSSTG22635', 'SDSSTG6159', 'SDSSTG9370', 'SDSSTG1601', 'SDSSTG28674'
, 'SDSSTG11320', 'SDSSTG8102', 'SDSSTG3669', 'SDSSTG11631', 'SDSSTG9178']

'SDSSTG40241'

In [ ]:
xspec_set1 = ['SDSSTG10842',
'SDSSTG10094','SDSSTG6159','SDSSTG8102','SDSSTG9178','SDSSTG39344']
xspec_set2 = ['SDSSTG5742','SDSSTG6058','SDSSTG1162','SDSSTG15354','SDSSTG2424'
,'SDSSTG46701','SDSSTG35976','SDSSTG9399','SDSSTG4436','SDSSTG9647']
xspec_set3 = ['SDSSTG4936','SDSSTG828','SDSSTG15776','SDSSTG9370','SDSSTG21128'
,'SDSSTG1398','SDSSTG40241','SDSSTG11844','SDSSTG4047','SDSSTG3513']
xspec_set4 = ['SDSSTG16393','SDSSTG16386','SDSSTG10159','SDSSTG11631','SDSSTG3669'
,'SDSSTG11320','SDSSTG28674','SDSSTG22635']

xspec_set_app1 = []
xspec_set_app2 = []

4.2 download images

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from glob import glob

my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/fk5_regs'
server_rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'

regname = 'R500-01'
# read ids
datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS/get_nh'
f = pd.read_csv(f'{datapath}/basics_allsources.csv')
ids = f['ID']

for id in tqdm(ids):
    print(id)
    savepath = f'{my_rootdir}/{id}'
    os.makedirs(savepath, exist_ok=True)
    FILE = f'R500-01_fk5.reg'
    files_exist = glob(f'{savepath}/fk5_regs/{FILE}')
    if not files_exist:
        os.system(f'scp -P3333 yanling@101.6.240.92:{server_rootdir}/{id}/{FILE} {savepath}/')
        # os.system(f'scp -P3333 yanling@101.6.240.92:{server_rootdir}/{id}/sources_final_sum_srcreg_fk5_edit.reg {savepath}/')
        # os.system(f'scp -P3333 yanling@101.6.240.92:{server_rootdir}/{id}/*-bkgregtsky.reg {savepath}/')

  0%|          | 0/48 [00:00<?, ?it/s]

SDSSTG8050


  2%|▏         | 1/48 [00:06<04:51,  6.20s/it]

SDSSTG10842


  4%|▍         | 2/48 [00:13<05:17,  6.89s/it]

...

 96%|█████████▌| 46/48 [04:38<00:12,  6.28s/it]

SDSSTG28674


 98%|█████████▊| 47/48 [04:45<00:06,  6.42s/it]

SDSSTG22635


100%|██████████| 48/48 [04:51<00:00,  6.07s/it]


4.3 rename files

In [ ]:
from glob import glob
import os

dirs = glob('SDSS*')
for d in dirs:
    files = glob(f'{d}/*R500*/*R500*/*SD*SD*')
    for f in files:
        newf = f.replace(f'{d}_{d}', f'{d}')
        print(newf)
        os.system(f'mv {f} {newf}')

In [ ]:
from glob import glob
import os

dirs = glob('IDSDSS*')
for d in dirs:
    newf = d.replace(f'IDSDSSTG', f'ID')
    print(newf)
    os.system(f'mv {d} {newf}')

In [ ]:
#!/bin/bash

# Iterate through files matching the pattern
for file in SDSSTG*R500-01*; do
    if [ -f "$file" ]; then
        # Extract the filename without the path
        filename=$(basename "$file")
        new_filename="${filename/R500/R500-}"

        # Move the file to the new name in the same directory
        mv "$file" "$new_filename"
    fi
done


4.4 upload rosat spectras

In [5]:
import pandas as pd
from glob import glob

# my_dir = f'/Users/eusracenorth/Downloads/ROSAT_specs'
my_dir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits'

my_rootdir = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/XGAP'
server_rootdir = '/data/yanling/XGAP-ABUN/data/alldata/XGAP'

# datapath = f'{my_rootdir}/../dom'
# f = pd.read_csv(f'{datapath}/source_centers_mos1.csv')
# ids = f['SRCID']

# # load the coordinates
# f = pd.read_csv(f'{my_rootdir}/../dom/my_centers.csv')
# ids = f['ID']

# regname = 'bkg'
# ids = ['SDSSTG10842']

datapath = f'/Users/eusracenorth/Documents/work/XGAP-ABUN/codes/ESAS/get_nh'
f = pd.read_csv(f'{datapath}/basics_allsources.csv')
ids = f['ID']

for id in ids:
    print(id)
    # srcname1 = f"ID{id.split('G')[-1]}"
    os.system(f'scp -P3333 {my_dir}/{id}/sources_final_sum_srcreg_fk5_reedit.reg yanling@101.6.240.92:{server_rootdir}/{id}')

SDSSTG8050
SDSSTG10842
SDSSTG885
SDSSTG1011
SDSSTG9695
SDSSTG4654


scp: stat local "/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits/SDSSTG9695/sources_final_sum_srcreg_fk5_reedit.reg": No such file or directory


SDSSTG1695
SDSSTG15641
SDSSTG3128
SDSSTG2620
SDSSTG12349


scp: stat local "/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits/SDSSTG2620/sources_final_sum_srcreg_fk5_reedit.reg": No such file or directory


SDSSTG16150
SDSSTG3460
SDSSTG9771
SDSSTG1601
SDSSTG10094
SDSSTG6159
SDSSTG8102
SDSSTG9178
SDSSTG39344
SDSSTG5742
SDSSTG6058
SDSSTG1162
SDSSTG15354
SDSSTG2424
SDSSTG46701
SDSSTG35976
SDSSTG9399
SDSSTG4436
SDSSTG9647
SDSSTG4936
SDSSTG828
SDSSTG15776
SDSSTG9370
SDSSTG21128


scp: stat local "/Users/eusracenorth/Documents/work/XGAP-ABUN/alldata/figs/comb_fits/SDSSTG9370/sources_final_sum_srcreg_fk5_reedit.reg": No such file or directory


SDSSTG1398
SDSSTG40241
SDSSTG11844
SDSSTG4047
SDSSTG3513
SDSSTG16393
SDSSTG16386
SDSSTG10159
SDSSTG11631
SDSSTG3669
SDSSTG11320
SDSSTG28674
SDSSTG22635
